In [22]:
import pandas as pd
import numpy as np

# GroupBy and Aggregates

This section of the course introduces the split-apply-combine pattern. We will first do this manually, and then learn the `grouby()` method. This method allows us to quickly slice and transform our dataset depending on the keys and logic that we provide.

We will then explore aggregation functions, grouping my multiple keys, and some advanced topics such as combining `groupby()` with additional methods, such as `transform()`, `filter()`, and `apply()`. This allows for highly customizable operations.

## New Data: Game Sales

In this section, we'll work with a dataset that contains sales information for the most popular games for the Xbox and Playstation consoles. It provides sales in millions of dollars. This dataset is originally from Kaggle, and has been modified by the instructor to enable the learnings of this section.

https://andybek.com/pandas-games

Let's start by importing the data

In [23]:
games_url = 'https://andybek.com/pandas-games'

In [24]:
games = pd.read_csv(games_url)

In [25]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          3143 non-null   object 
 1   Platform      3143 non-null   object 
 2   Year          3088 non-null   float64
 3   Genre         3143 non-null   object 
 4   Publisher     3136 non-null   object 
 5   NA_Sales      3143 non-null   float64
 6   EU_Sales      3143 non-null   float64
 7   JP_Sales      3143 non-null   float64
 8   Other_Sales   3143 non-null   float64
 9   Global_Sales  3143 non-null   float64
dtypes: float64(6), object(4)
memory usage: 245.7+ KB


In [26]:
games.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,14.97,4.94,0.24,1.67,21.82
1,Grand Theft Auto V,PS3,2013.0,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
2,Grand Theft Auto V,X360,2013.0,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38
3,Call of Duty: Modern Warfare 3,X360,2011.0,Shooter,Activision,9.03,4.28,0.13,1.32,14.76
4,Call of Duty: Black Ops,X360,2010.0,Shooter,Activision,9.67,3.73,0.11,1.13,14.64


We've got some information on the game name, the console ("Platform"), year of release, genre of game, the game publisher, and sales in different countries as well as overall global sales. Let's get to it!

In [27]:
games["Platform"].unique()

array(['X360', 'PS3', 'PS4', 'XOne'], dtype=object)

## Simple Aggregations Overview

In previous sections, we used simple aggregation functions to calculate things like mean, sum, standard deviation, variance, etc. Let's review in this lecture, in preparation for the `groupby()` method.

Let's first answer "what are the total sales across ALL regions". One way to solve this is to isolate the columns of interest using the `loc[]` indexer.

In [28]:
games.loc[:, ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,14.97,4.94,0.24,1.67
1,7.01,9.27,0.97,4.14
2,9.63,5.31,0.06,1.38
3,9.03,4.28,0.13,1.32
4,9.67,3.73,0.11,1.13
...,...,...,...,...
3138,0.00,0.01,0.00,0.00
3139,0.01,0.00,0.00,0.00
3140,0.01,0.00,0.00,0.00
3141,0.00,0.01,0.00,0.00


We've created a small dataframe slice with just the sales columns. Now we can chain on a `sum()` function to get the sums of all of the columns.

In [29]:
games.loc[:, ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].sum()

NA_Sales       1173.30
EU_Sales        793.64
JP_Sales        107.06
Other_Sales     282.75
dtype: float64

Notice that the dimensionality changed when we applied the aggregate `sum()` function. What began as a dataframe of 3134 rows and 4 columns ended up as a series with four entries. This is a  feature of aggregate functions in general.

In [30]:
games.loc[:, ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].median()

NA_Sales       0.14
EU_Sales       0.07
JP_Sales       0.00
Other_Sales    0.03
dtype: float64

In [31]:
games.loc[:, ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].std()

NA_Sales       0.801483
EU_Sales       0.570164
JP_Sales       0.093115
Other_Sales    0.203866
dtype: float64

One thing to note about aggregation: it is technically an operation that can be applied in more ways than one. For example, any give column can be collapsed or aggregated across all of its rows. Similarly, any row can be aggregated across all of its columns.

When dealing with dataframes, Pandas defaults to the vertical aggregation. We can change this by changing the axis parameter of the aggregate function. For instance, let's perform the standard deviation calculation along the column axis.

In [32]:
games.loc[:, ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']].std(axis = 1)

0       6.641358
1       3.594926
2       4.311415
3       3.964602
4       4.286739
          ...   
3138    0.005000
3139    0.005000
3140    0.005000
3141    0.005000
3142    0.005000
Length: 3143, dtype: float64

Notice now that when changing the axis of aggregation, we now have a series with 3143 entries. We're getting the standard deviation of the sales across the four regions for each individual row (video game).

So two quick takeaways:
1. When we apply aggregation functions, the dimensions of the data frequently change.
2. We can aggregate either horizontally or vertically.

## Conditional Aggregates

The aggregates that we computed in the previous lecture applied across the entire column or row. Sometimes however, we only want to apply the function to a smaller set of values within the dataframe.

Suppose we want the total sales by region, but only for particular platforms/consoles. How do we do that?

In [33]:
games.Platform.unique()

array(['X360', 'PS3', 'PS4', 'XOne'], dtype=object)

Let's go with the Xbox 360 and PS3 platforms. Find the total sales by region for the X360 and PS3.

Let's start by creating a new view that only includes the Platform and Sales columns.

In [34]:
sales = games.loc[:, ['Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]

In [35]:
sales.head()

,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,X360,14.97,4.94,0.24,1.67
1,PS3,7.01,9.27,0.97,4.14
2,X360,9.63,5.31,0.06,1.38
3,X360,9.03,4.28,0.13,1.32
4,X360,9.67,3.73,0.11,1.13


How do we segregate the platforms? If we were to apply the `sum()` function here directly, we'll get the sum across both platforms.

In [36]:
sales.sum(numeric_only=True)

NA_Sales       1173.30
EU_Sales        793.64
JP_Sales        107.06
Other_Sales     282.75
dtype: float64

That's not what we want. Instead, we want *platform-specific* sales for each region. To do that, we'll need to be more specific to our data selection. 

One thing we can do is use label-based conditional indexing, selecting only for the platform we want, and then summing for that platform.

In [37]:
sales.loc[sales.Platform == 'X360']

,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,X360,14.97,4.94,0.24,1.67
2,X360,9.63,5.31,0.06,1.38
3,X360,9.03,4.28,0.13,1.32
4,X360,9.67,3.73,0.11,1.13
7,X360,8.25,4.30,0.07,1.12
...,...,...,...,...,...
3127,X360,0.00,0.01,0.00,0.00
3128,X360,0.00,0.00,0.01,0.00
3130,X360,0.01,0.00,0.00,0.00
3135,X360,0.00,0.00,0.01,0.00


In [38]:
sales.loc[sales.Platform == 'X360'].sum(numeric_only=True)

NA_Sales       601.05
EU_Sales       280.58
JP_Sales        12.43
Other_Sales     85.54
dtype: float64

We can do the same for PS3.

In [39]:
sales.loc[sales.Platform == 'PS3'].sum(numeric_only=True)

NA_Sales       392.26
EU_Sales       343.71
JP_Sales        79.99
Other_Sales    141.93
dtype: float64

Simple enough, but rather verbose and repetitive. What if we wanted to do it in a more eloquent manner with less code? Moreover, what if we had dozens or hundreds of platforms that we wanted to do an aggregate analysis for? This manual platform-by-platform approach is not practical for that. We would need a better approach. And there is such an approach. Stay tuned!

## The Split-Apply-Combine Pattern

Before we get into the `groupby()` method, let's conceptually review what a data analysis pattern that we've been applying.

In the previous lecture, we wanted to calculate the sum of sales for all regions for two specific platforms - the X360 and the PS3. We could not just simply apply `sum()` to the dataframe because the resulting total gave us the total sales across all platforms.

To address this issue, we partitioned the data into separate groups and created smaller dataframes containing data from only the platforms we cared about. Once we had those datasets, we applied aggregation functions to calculate our sums. This approach is known as the **split-apply-combine** pattern, coined by Hadley Wickham from Rice University.

The main idea is to break down a big problem into smaller pieces, solve those pieces, and then put it all back together at the end. We've covered the first two steps - split and apply. We have not yet seen combine, but we will! In fact, the `groupby()` method which we will see in the next method implements ALL THREE of these steps in one single operation. Totally awesome.

## Introducting the `groupby()` Method

Everything and more that we have been doing manually earlier in this section can be done by the `groupby()` method. It is essentially a pivot table for Pandas, which applies a split, apply, combine operation in one go.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

Let's tackle that platform-specific sales question again. All we need to do is identify what we want to group by ("Platform" in this case), and which aggregate function we want to apply!

In [40]:
sales.groupby("Platform").sum()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Platform,,,,
PS3,392.26,343.71,79.99,141.93
PS4,96.80,123.70,14.30,43.36
X360,601.05,280.58,12.43,85.54
XOne,83.19,45.65,0.34,11.92


Man that was so easy, and much better than the manual work we had to do earlier. Even better, it combined all of the output in the end. The `groupby()` method is extremely powerful, and there will be many instances in our data analysis journey where we'll want to use it.

We can apply essentially any function that we want. For example, we can find the average sales across all regions for each platform.

In [42]:
sales.groupby("Platform").mean()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Platform,,,,
PS3,0.295154,0.258623,0.060188,0.106795
PS4,0.288095,0.368155,0.042560,0.129048
X360,0.475138,0.221802,0.009826,0.067621
XOne,0.390563,0.214319,0.001596,0.055962


It looks like the Xbox 360 sells far more in North American than the other platforms. But the average is only one side of the story. Perhaps this value is skewed by some very high-selling games. We can investigate by looking at the median.

In [43]:
sales.groupby("Platform").median()

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Platform,,,,
PS3,0.12,0.07,0.01,0.03
PS4,0.07,0.08,0.02,0.03
X360,0.17,0.06,0.00,0.02
XOne,0.15,0.07,0.00,0.02


We see that the median North American sales for the Xbox 360 is less than half of the mean, indicating that the majority of the sales are below-average and that the average is driven upward by a few games that sell very well. 

Stay tuned, we'll later see more advanced operations such as splitting by multiple keys.

## The DataFrameGroupBy Object

When using `groupby()`, you create what's known as a **DataFrameGroupBy** object. Let's take a look under the hood here.

What happens if we just perform the `groupby()` but do not apply a method/function to it?

In [44]:
sales.groupby("Platform")

What returns is a DataFrameGroupby object and a reference to its place in memory. Think of this as a special view into an intermediate output. At this point, Pandas has validated our mapping based on the key that we want to group by, and is awaiting instructions on what to do next. The split has not yet happened, but it will once Pandas knows what to do.

There are four platforms that we are grouping by, and thus the length of the DataFrameGroupBy object is 4.

In [46]:
len(sales.groupby("Platform"))

4

As soon as we apply a function to this object, the computation will kick in. So the key thing to remember here is that applying `groupby()` on a dataframe returns this DataFrameGroupBy object. It does *not* return a dataframe.